# Importy

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import kagglehub
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

# Przygotowanie danych

In [ ]:
# Pobranie datasetu
path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")
metadata_path = os.path.join(path[0], 'HAM10000_metadata.csv')
df = pd.read_csv(metadata_path)

# Przygotowanie ścieżek do obrazów
images_path = os.path.join(path[0], 'images')
df['filepath'] = df['image_id'].apply(lambda x: os.path.join(images_path, f'{x}.jpg'))

# Podział na zbiory treningowy, walidacyjny i testowy (60-20-20)
train_df, temp_df = train_test_split(df, test_size=0.4, stratify=df['dx'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['dx'], random_state=42)

print(f"Rozmiar zbioru treningowego: {len(train_df)}")
print(f"Rozmiar zbioru walidacyjnego: {len(val_df)}")
print(f"Rozmiar zbioru testowego: {len(test_df)}")

# Parametry
img_size = (224, 224)
batch_size = 32

# Generator dla treningu z augmentacją
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generator dla walidacji i testu (tylko skalowanie)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Generatory danych
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='dx',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filepath',
    y_col='dx',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepath',
    y_col='dx',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Przykładowe zdjecia

In [1]:
import matplotlib.pyplot as plt
from PIL import Image

# Słownik z opisami klas
dx_descriptions = {
    'akiec': 'Rogowacenie słoneczne / Rak kolczystokomórkowy (in situ)',
    'bcc': 'Rak podstawnokomórkowy',
    'bkl': 'Łagodne zmiany keratotyczne (keratosis benigna)',
    'df': 'Włókniak twardy (dermatofibroma)',
    'nv': 'Znamię melanocytowe',
    'mel': 'Czerniak',
    'vasc': 'Zmiany naczyniowe'
}

# Pobierz po jednym przykładzie z każdej klasy
examples = {}
for dx in df['dx'].unique():
    example = df[df['dx'] == dx].iloc[0]
    examples[dx] = example['filepath']

# Utwórz wykres
plt.figure(figsize=(20, 10))

for idx, (dx, img_path) in enumerate(examples.items(), 1):
    try:
        img = Image.open(img_path)
        plt.subplot(2, 4, idx)
        plt.imshow(img)
        plt.title(f"{dx.upper()}\n{dx_descriptions[dx]}", fontsize=10)
        plt.axis('off')
    except Exception as e:
        print(f"Nie udało się załadować obrazu dla klasy {dx}: {e}")

plt.tight_layout()
plt.show()

# Wyświetl statystyki dla każdej klasy
print("\nLiczba przypadków w każdej klasie:")
class_counts = df['dx'].value_counts()
for dx in df['dx'].unique():
    print(f"\n{dx.upper()} - {dx_descriptions[dx]}")
    print(f"Liczba przypadków: {class_counts[dx]}")

NameError: name 'df' is not defined

# Model 1: Baseline CNN

In [ ]:
print("Trenowanie modelu CNN od podstaw...")

model_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

model_cnn.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    mode='min',
    verbose=1
)

model_cnn.summary()

history_cnn = model_cnn.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=50,
    callbacks=[early_stopping]
)

# Zapisz model
model_cnn.save('models/model_cnn.h5')

Trenowanie modelu CNN od podstaw...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,863 (42.61 MB)

 Trainable params: 11,169,863 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

d:\Anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 264s 1s/step - accuracy: 0.6585 - loss: 1.1482 - val_accuracy: 0.6704 - val_loss: 0.9911
Epoch 2/25
  1/250 ━━━━━━━━━━━━━━━━━━━━ 1:33 375ms/step - accuracy: 0.6250 - loss: 0.9870

d:\Anaconda\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 65ms/step - accuracy: 0.6250 - loss: 0.9870 - val_accuracy: 0.6694 - val_loss: 0.9929
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 188s 753ms/step - accuracy: 0.6634 - loss: 1.0165 - val_accuracy: 0.6673 - val_loss: 0.9183
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 65ms/step - accuracy: 0.6875 - loss: 0.7565 - val_accuracy: 0.6668 - val_loss: 0.9205
Epoch 5/25
 73/250 ━━━━━━━━━━━━━━━━━━━━ 2:03 700ms/step - accuracy: 0.6488 - loss: 0.9581

# Wizualizacja wyników dla modelu CNN

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history_cnn.history['loss'], label='Strata treningowa')
plt.plot(history_cnn.history['val_loss'], label='Strata walidacyjna')
plt.title('Loss - CNN')
plt.xlabel('Epoka')
plt.ylabel('Strata')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_cnn.history['accuracy'], label='Dokładność treningowa')
plt.plot(history_cnn.history['val_accuracy'], label='Dokładność walidacyjna')
plt.title('Accuracy - CNN')
plt.xlabel('Epoka')
plt.ylabel('Dokładność')
plt.legend()

plt.tight_layout()
plt.show()

# Macierz pomyłek dla modelu CNN

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
from tensorflow.keras.models import load_model

# Wczytaj zapisany model
model_cnn = load_model('models/model_cnn.h5')

# Przewidywania dla zbioru testowego
y_true = test_generator.classes
y_pred = model_cnn.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Macierz pomyłek
cm = confusion_matrix(y_true, y_pred_classes)
cmd = ConfusionMatrixDisplay(cm, display_labels=test_generator.class_indices.keys())
cmd.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix - CNN')
plt.show()

# Procentowa macierz pomyłek
cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
cmd_percentage = ConfusionMatrixDisplay(cm_percentage, display_labels=test_generator.class_indices.keys())
cmd_percentage.plot(cmap=plt.cm.Blues)
plt.title('Percentage Confusion Matrix - CNN')
plt.show()

NameError: name 'test_generator' is not defined

# Model 2: Transfer Learning z MobileNetV2

In [ ]:
print("Trenowanie modelu z Transfer Learningiem (MobileNetV2)...")

# Wczytanie pretrenowanego modelu MobileNetV2 (bez głowy klasyfikacyjnej)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))
base_model.trainable = False  # Zamrażamy warstwy MobileNetV2

model_tl = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Redukcja wymiarów wyjścia z MobileNetV2
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

model_tl.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model_tl.summary()

history_tl = model_tl.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=25
)

# Zapisz model
model_tl.save('models/model_tl.h5')

# Wizualizacja wyników dla modelu z transfer learningiem

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history_tl.history['loss'], label='Strata treningowa')
plt.plot(history_tl.history['val_loss'], label='Strata walidacyjna')
plt.title('Loss - Transfer Learning')
plt.xlabel('Epoka')
plt.ylabel('Strata')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_tl.history['accuracy'], label='Dokładność treningowa')
plt.plot(history_tl.history['val_accuracy'], label='Dokładność walidacyjna')
plt.title('Accuracy - Transfer Learning')
plt.xlabel('Epoka')
plt.ylabel('Dokładność')
plt.legend()

plt.tight_layout()
plt.show()

# Macierz pomyłek dla modelu z transfer learningiem

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
from tensorflow.keras.models import load_model

# Wczytaj zapisany model
model_tl = load_model('models/model_tl.h5')

# Przewidywania dla zbioru testowego
y_true_tl = test_generator.classes
y_pred_tl = model_tl.predict(test_generator)
y_pred_classes_tl = np.argmax(y_pred_tl, axis=1)

# Macierz pomyłek
cm_tl = confusion_matrix(y_true_tl, y_pred_classes_tl)
cmd_tl = ConfusionMatrixDisplay(cm_tl, display_labels=test_generator.class_indices.keys())
cmd_tl.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix - Transfer Learning')
plt.show()

# Procentowa macierz pomyłek
cm_percentage_tl = cm_tl.astype('float') / cm_tl.sum(axis=1)[:, np.newaxis] * 100
cmd_percentage_tl = ConfusionMatrixDisplay(cm_percentage_tl, display_labels=test_generator.class_indices.keys())
cmd_percentage_tl.plot(cmap=plt.cm.Blues)
plt.title('Percentage Confusion Matrix - Transfer Learning')
plt.show()